<a href="https://colab.research.google.com/github/anitayadav3/EmotionRecognitionInConversation/blob/master/ALBERT_XLarge_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install tensorflow_text
import tensorflow_text as text 

     |████████████████████████████████| 3.4MB 17.6MB/s 


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, GRU, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import pickle
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import string
import requests

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
preprocessor = hub.KerasLayer("https://hub.tensorflow.google.cn/tensorflow/albert_en_preprocess/2")
encoder = hub.KerasLayer("https://hub.tensorflow.google.cn/tensorflow/albert_en_xlarge/2",trainable=True)

In [7]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
encoder_inputs = preprocessor(text_input)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 2048].
out = Dense(100, activation='relu')(pooled_output)
out = Dense(100, activation='relu')(out)
out = Dense(6, activation='softmax')(out)

In [8]:
L=[]
model = tf.keras.Model(text_input, out)
sentences = tf.constant(["My text"])
print((model(sentences)))

tf.Tensor([[0.5728689  0.10440885 0.10842568 0.06997757 0.0524542  0.09186476]], shape=(1, 6), dtype=float32)


In [11]:
def build_model(bert_layer):
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
  encoder_inputs = preprocessor(text_input)
  outputs = bert_layer(encoder_inputs)
  pooled_output = outputs["pooled_output"]      # [batch_size, 2048].
  out = Dense(100, activation='relu')(pooled_output)
  out = Dense(100, activation='relu')(out)
  out = Dense(6, activation='softmax')(out)
  model = Model(inputs=text_input, outputs=out)
  model.compile(Adam(lr=2e-6), loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [12]:
model = build_model(encoder)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_word_ids': ( 0           input_3[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'encoder_outputs':  58724864    keras_layer[2][0]                
                                                                 keras_layer[2][1]                
                                                                 keras_layer[2][2]                
____________________________________________________________________________________________